In [ ]:
import syft as sy
sy.requires(">=0.8.1-beta")
sy.enable_autoreload()

In [ ]:
domain_1 = sy.orchestra.launch(name="test-domain-1", dev_mode=True, reset=True)

In [ ]:
domain_1_client = domain_1.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_2 = sy.orchestra.launch(name="test-domain-2", dev_mode=True, reset=True)

In [ ]:
domain_2_client = domain_2.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_1_client.exchange_route(domain_2_client)

In [ ]:
domain_1_client.domains

In [ ]:
assert len(domain_1_client.domains) == 1

In [ ]:
domain_2_client.domains

In [ ]:
assert len(domain_2_client.domains) == 1

In [ ]:
from syft.service.project.project import NewProjectSubmit
from syft.service.project.project import NewProject
from syft.service.project.project import DemocraticConsensusModel
from syft.service.metadata.node_metadata import NodeMetadata

In [ ]:
project = NewProjectSubmit(
    name="test project",
    description="my test project",
    shareholders=[domain_1_client, domain_2_client],
    consensus_model=DemocraticConsensusModel(),
)
project

In [ ]:
results = project.start()
results

In [ ]:
results[0].start_hash

In [ ]:
assert results[0].start_hash == results[1].start_hash

In [ ]:
project = results[0]

In [ ]:
project.shareholders

In [ ]:
assert isinstance(project, NewProject)

In [ ]:
assert len(domain_1_client.api.services.newproject.get_all()) == 1

## Project Messaging

In [ ]:
project = domain_1_client.api.services.newproject.get_all()[0]

In [ ]:
from syft.service.project.project import ProjectMessage

In [ ]:
project.events

In [ ]:
message = "My cool message"

message_event = ProjectMessage(
    message=message
)

In [ ]:
result = message_event.publish(project, domain_1_client.credentials)
result

In [ ]:
# duplicates are not an error when using .publish
result = message_event.publish(project, domain_1_client.credentials)
assert isinstance(result, sy.SyftSuccess)
result

In [ ]:
# or another way to do it
try:
    result = project.add_event(message_event, domain_1_client)
except Exception as e:
    print(e)

In [ ]:
assert len(project.events) == 1

In [ ]:
project.events[0]

In [ ]:
assert project.events[0].project_id == project.id
assert project.events[0].prev_event_uid == project.id
assert project.events[0].prev_signed_event_hash == project.start_hash

In [ ]:
domain_1_client.api.services.newproject.get_all()[0]

In [ ]:
domain_2_client.api.services.newproject.get_all()[0]

In [ ]:
try:
    result = project.add_event(message_event, domain_1_client)
except Exception as e:
    print(e)

In [ ]:
assert project.validate_events()

## Dataset Upload

In [ ]:
# import numpy as np
# dataset = sy.Dataset(name="Domains's... Private Data")
# asset = sy.Asset(name="Secret data")

# # Real Data
# x = np.array([1, 2, 3])
# asset.set_obj(x)

# # Mock Data
# y = np.array([1, 1, 1])
# asset.set_mock(y, mock_is_real=False)
# dataset.add_asset(asset)

In [ ]:
# domain_1_client.upload_dataset(dataset)

In [ ]:
# domain_1_client.datasets

In [ ]:
# domain_1_dataset = domain_1_client.datasets[-1]
# @sy.syft_function(
#     input_policy=sy.ExactMatch(
#         input_data=domain_1_dataset.assets[0]
#     ),
#     output_policy=sy.SingleExecutionExactOutput(),
# )
# def simple_function(input_data):

#     return input_data+input_data

In [ ]:
# simple_function

In [ ]:
# req = domain_1_client.api.services.code.request_code_execution(simple_function)

In [ ]:
# updated_proj = project.add_event(message_event)

In [ ]:
# updated_proj

In [ ]:
# domain_1_client.api.services.newproject.get_all()[0]

In [ ]:
# domain_2_client.api.services.newproject.get_all()[0]

In [ ]:
for node in [domain_1, domain_2]:
    if node.node_type.value == "python":
        node.land()